In [1]:
import numpy as np
import scipy

In [2]:
from model import Network
from data import load_mnist_dataloaders
import torch
import torchvision.transforms as transforms
import torch.nn as nn

In [3]:
# load to model
model_config = './model_config/fc1_cocktail_party_mnist_instance.json'
checkpoint_path = './checkpoints/122123_fc1_cocktail_party_mnist_pretraining_wout_bias.pth'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Network(model_config)
checkpoint = torch.load(checkpoint_path)
model.load_state_dict(checkpoint['state_dict'])
model = model.to(device)

# get val loader
normalize_mean, normalize_std = (0.1307,), (0.3081,)
batch_size = 8
data_path = './data'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(normalize_mean, normalize_std),
])

_, val_dataloader = load_mnist_dataloaders(data_path, batch_size, transform)
selected_val_batch_data, selected_val_batch_label = next(iter(val_dataloader))
selected_val_batch_data = selected_val_batch_data.to(device)
selected_val_batch_label = selected_val_batch_label.to(device)

In [4]:
# receiving gradients
model.zero_grad()
criterion = nn.CrossEntropyLoss()
output = model(selected_val_batch_data.reshape(batch_size, -1))
loss = criterion(output, selected_val_batch_label)
loss.backward()
gradient_of_layers = []
for param in model.parameters():
    gradient_of_layers.append(param.grad.data.clone().to('cpu'))
print([x.size() for x in gradient_of_layers])

[torch.Size([10, 784])]


In [5]:
def demean(X):
    # Note that this is the same as J = (I - 11^T/n)
    ones = np.ones((X.shape[0], 1))
    J = np.identity(X.shape[0]) - ((np.matmul(ones, ones.T)) / X.shape[0])
    return np.matmul(J, X)

def project(X, V_trans):
    # Note that this is the same as V
    return np.matmul(X, V_trans.T)

def scale(X, S):
    # Note that this is the same as S^{-1}
    return np.multiply(X, (1 / S))

def unrotate(X, V):
    # Note that this is the same as V^{T}
    return np.matmul(X, V)

In [6]:
gradient = demean(gradient_of_layers[0].detach().numpy().T)
print(gradient.shape)

(784, 10)


In [7]:
U, S, V_trans = np.linalg.svd(gradient)

In [8]:
print(S.shape)

(10,)


In [9]:
x_proj = project(gradient, V_trans)

In [10]:
x_scaled = scale(x_proj, S)

In [11]:
x_unrotated = unrotate(x_scaled, V_trans)

In [12]:
whitened_gradient = x_unrotated.T

In [13]:
gradient

array([[-4.57331351e-05, -1.51589345e-05, -2.25460471e-03, ...,
        -1.05363532e-02,  1.57396371e-02,  4.67932431e-03],
       [-4.57331351e-05, -1.51589345e-05, -2.25460471e-03, ...,
        -1.05363532e-02,  1.57396371e-02,  4.67932431e-03],
       [-4.57331351e-05, -1.51589345e-05, -2.25460471e-03, ...,
        -1.05363532e-02,  1.57396371e-02,  4.67932431e-03],
       ...,
       [-4.57331351e-05, -1.51589345e-05, -2.25460471e-03, ...,
        -1.05363532e-02,  1.57396371e-02,  4.67932431e-03],
       [-4.57331351e-05, -1.51589345e-05, -2.25460471e-03, ...,
        -1.05363532e-02,  1.57396371e-02,  4.67932431e-03],
       [-4.57331351e-05, -1.51589345e-05, -2.25460471e-03, ...,
        -1.05363532e-02,  1.57396371e-02,  4.67932431e-03]])

In [14]:
whitened_gradient = torch.from_numpy(whitened_gradient).to(torch.float32)

In [15]:
whitened_gradient

tensor([[-0.0014, -0.0014, -0.0014,  ..., -0.0014, -0.0014, -0.0014],
        [ 0.0033,  0.0033,  0.0033,  ...,  0.0033,  0.0033,  0.0033],
        [-0.0033, -0.0033, -0.0033,  ..., -0.0033, -0.0033, -0.0033],
        ...,
        [-0.0094, -0.0094, -0.0094,  ..., -0.0094, -0.0094, -0.0094],
        [ 0.0087,  0.0087,  0.0087,  ...,  0.0087,  0.0087,  0.0087],
        [ 0.0076,  0.0076,  0.0076,  ...,  0.0076,  0.0076,  0.0076]])

In [16]:
from criterion import ReconstructImageFromFCLoss
# criterion output testing
unmixing_matrix = torch.rand((selected_val_batch_data.size(0), gradient_of_layers[0].size(0)), requires_grad=True)
reconstruction_loss = ReconstructImageFromFCLoss(28, 28, 1, 1, 1, 1)
optimizer = torch.optim.Adam([unmixing_matrix])

for iter_idx in range(25000):
    optimizer.zero_grad()
    # out_score, non_gaussianity_score, total_variance_score, mutual_independence_score
    loss, _, _, _ = reconstruction_loss(unmixing_matrix, whitened_gradient)
    loss.backward()
    optimizer.step()
    
    if (iter_idx + 1) % 1000 == 0 or iter_idx == 0:
        print('loss: {}'.format(loss.item()))

loss: 1.8421250581741333
loss: 0.9218711853027344
loss: 0.9133086204528809
loss: 0.9053990244865417
loss: 0.8990450501441956
loss: 0.8946385979652405
loss: 0.8911397457122803
loss: 0.8884327411651611
loss: 0.8865076303482056
loss: 0.8848778009414673
loss: 0.8836724758148193
loss: 0.8827620148658752
loss: 0.8820016384124756
loss: 0.8820883631706238
loss: 0.8813413381576538
loss: 0.8811435103416443
loss: 0.8807217478752136
loss: 0.8806502819061279
loss: 0.880102813243866
loss: 0.8802399635314941
loss: 0.8803988099098206
loss: 0.8799233436584473
loss: 0.8803843855857849
loss: 0.8804811835289001
loss: 0.8804382085800171
loss: 0.8805578351020813


In [17]:
with torch.no_grad():
    estimated_img = unmixing_matrix @ whitened_gradient
    for estimate in estimated_img:
        img = transforms.ToPILImage()(torch.clamp(estimate.reshape(1, 28, 28), min=-1, max=1))
        # imshow(np.asarray(img))
        img.show()


(eog:24851): EOG-CRITICAL **: 15:23:34.826: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:24851): GLib-GIO-CRITICAL **: 15:23:34.826: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:24851): EOG-CRITICAL **: 15:23:34.844: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:24851): GLib-GIO-CRITICAL **: 15:23:34.844: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:24851): EOG-CRITICAL **: 15:23:34.904: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:24851): GLib-GIO-CRITICAL **: 15:23:34.904: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:24851): EOG-CRITICAL **: 15:23:34.904: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:24851): GLib-GIO-CRITICAL **: 15:23:34.904: g_file_equal: assertion 'G_IS_FILE (file1)' failed

(eog:24851): EOG-CRITICAL **: 15:23:34.956: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:24851): GLib-GIO-CRITICAL **: 15:23:34.956: g_file_equal: assertion 'G_IS_F

In [31]:
with torch.no_grad():
    # estimated_img = unmixing_matrix @ whitened_gradient
    for img_arr in selected_val_batch_data:
        img = transforms.ToPILImage()(torch.clamp(img_arr.reshape(1, 28, 28), min=-1, max=1))
        # imshow(np.asarray(img))
        img.show()